# Cox-PH and DeepSurv

In this notebook we will train the [Cox-PH method](http://jmlr.org/papers/volume20/18-424/18-424.pdf), also known as [DeepSurv](https://bmcmedresmethodol.biomedcentral.com/articles/10.1186/s12874-018-0482-1).
This notebook is modified from [the Cox-PH GitHub Repo](https://github.com/havakv/pycox?tab=readme-ov-file#methods).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

import torch
import torchtuples as tt

from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [ ]:
## Uncomment to install `sklearn-pandas`
! pip install sklearn-pandas

In [ ]:
np.random.seed(1234)
_ = torch.manual_seed(123)

## Upload dataset

In [ ]:
data = pd.read_csv("../clean_data/nafl/combined.large.nafl.csv")

X = data.drop(columns=['DaysUntilFirstProgression', 'Outcome'])
Y = data[['StudyID', 'DaysUntilFirstProgression', 'Outcome']]

X = X.set_index('StudyID')
Y = Y.set_index('StudyID')

# convert categorical variables to one-hot variables
Y['Outcome'] = Y['Outcome'].astype(int)

In [ ]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X_torch, Y_torch, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

## Feature transforms

In [ ]:
# using the dataframe mapper method to standardize
cols_numerical = [x for x in X.columns if any(keyword in x for keyword in ['BMI', 'Age', 'Lab'])]
cols_other = [x for x in X.columns if x not in cols_numerical]

standardize = [([col], StandardScaler()) for col in cols_numerical]
leave = [(col, None) for col in cols_other]

x_mapper = DataFrameMapper(standardize + leave)

In [ ]:
x_train = x_mapper.fit_transform(X_train).astype('float32')
x_val = x_mapper.transform(X_val).astype('float32')
x_test = x_mapper.transform(X_test).astype('float32')

In [ ]:
get_target = lambda df: (df['DaysUntilFirstProgression'].values, df['Outcome'].values)
y_train = get_target(y_train)
y_val = get_target(y_val)
durations_test, events_test = get_target(y_test)
val = x_val, y_val

In [ ]:
# convert to the correct datatypes
y_train_new = (y_train[0].astype('float32'), y_train[1].astype('int32'))
y_val_new = (y_val[0].astype('float32'), y_val[1].astype('int32'))
durations_test = durations_test.astype('float32')
events_test = events_test.astype('int32')

## Neural net

We create a simple MLP with two hidden layers, ReLU activations, batch norm and dropout. 
Here, we just use the `torchtuples.practical.MLPVanilla` net to do this.

Note that we set `out_features` to 1, and that we have not `output_bias`.

In [ ]:
in_features = x_train.shape[1]
num_nodes = [32, 32]
out_features = 1
batch_norm = True
dropout = 0.1
output_bias = False

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                              dropout, output_bias=output_bias)

## Training the model

To train the model we need to define an optimizer. You can choose any `torch.optim` optimizer, but here we instead use one from `tt.optim` as it has some added functionality.
We use the `Adam` optimizer, but instead of choosing a learning rate, we will use the scheme proposed by [Smith 2017](https://arxiv.org/pdf/1506.01186.pdf) to find a suitable learning rate with `model.lr_finder`. See [this post](https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6) for an explanation.

In [ ]:
model = CoxPH(net, tt.optim.Adam)

In [ ]:
batch_size = 256
lrfinder = model.lr_finder(x_train, y_train_new, batch_size, tolerance=10)
_ = lrfinder.plot()
# plt.savefig("batch_loss_learning_rate.png", dpi=300)

In [ ]:
lrfinder.get_best_lr()

We include the `EarlyStopping` callback to stop training when the validation loss stops improving. After training, this callback will also load the best performing model in terms of validation loss.

In [ ]:
epochs = 512
callbacks = [tt.callbacks.EarlyStopping()]
verbose = True

In [ ]:
%%time
log = model.fit(x_train, y_train_new, batch_size, epochs, callbacks, verbose,
                val_data=val, val_batch_size=batch_size)

In [ ]:
_ = log.plot()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Training Epochs')
# plt.savefig("loss_per_epoch.png", dpi=300)

We can get the partial log-likelihood

In [ ]:
model.partial_log_likelihood(*val).mean()

In [ ]:
# get training PLL
train = x_train, y_train_new
model.partial_log_likelihood(*train).mean()

## Prediction

For evaluation we first need to obtain survival estimates for the test set.
This can be done with `model.predict_surv` which returns an array of survival estimates, or with `model.predict_surv_df` which returns the survival estimates as a dataframe.

However, as `CoxPH` is semi-parametric, we first need to get the non-parametric baseline hazard estimates with `compute_baseline_hazards`. 

Note that for large datasets the `sample` argument can be used to estimate the baseline hazard on a subset.

In [ ]:
_ = model.compute_baseline_hazards()

In [ ]:
surv = model.predict_surv_df(x_test)

In [ ]:
y_test_studyid = y_test.reset_index(inplace=False)
pos_outcome = y_test_studyid.index[y_test['Outcome'] == 1].tolist()
neg_outcome = y_test_studyid.index[y_test['Outcome'] == 0].tolist()

In [ ]:
# calculate one curve for all patients with positive progression
pos_surv = surv[pos_outcome]
pos_surv_avg = pos_surv.mean(axis=1)

In [ ]:
# calculate one curve for all patients with negative progression
neg_surv = surv[neg_outcome]
neg_surv_avg = neg_surv.mean(axis=1)

## Evaluation

We can use the `EvalSurv` class for evaluation the concordance, brier score and binomial log-likelihood. Setting `censor_surv='km'` means that we estimate the censoring distribution by Kaplan-Meier on the test set.

In [ ]:
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

In [ ]:
ev.concordance_td()

In [ ]:
time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
_ = ev.brier_score(time_grid).plot()

# plt.figure(figsize=(6.4, 4.8))
plt.xlabel('Time t (days)')
plt.ylabel('Brier Score')
plt.ylim(0, 0.9)
plt.title('Cox PH DeepSurv Brier Score Over Time')
# plt.savefig("results/deepsurv_brier_score.png", dpi=300)

In [ ]:
ev.integrated_brier_score(time_grid)

In [ ]:
ev.integrated_nbll(time_grid)

In [ ]:
# perform the same analysis for the training dataset

surv_train = model.predict_surv_df(x_train)

ev_train = EvalSurv(
    surv_train,
    durations=y_train_new[0],
    events=y_train_new[1],
    censor_surv='km'
)

In [ ]:
ev_train.concordance_td()

In [ ]:
ev_train.integrated_brier_score(time_grid)

In [ ]:
ev_train.integrated_nbll(time_grid)

# Adapt to produce SHAP scores

In [ ]:
import shap
import torch

net.eval()
net.to('cpu')

# DeepExplainer works directly with the PyTorch model
explainer = shap.DeepExplainer(net, torch.tensor(x_train, dtype=torch.float32))

In [ ]:
shap_values = explainer.shap_values(torch.tensor(x_test, dtype=torch.float32))

In [ ]:
shap_values_squeezed = shap_values.squeeze(-1)
shap_values_squeezed.shape

In [ ]:
# plot with human readable names
features = ['Transaminase-SGOT AST Blood Test', 'Iron Binding Capacity Test', 'Chest pain', 'Essential (primary) hypertension', 'Hemoglobin Blood Test', 'Very low-density lipoprotein Blood Test', 'Type 2 diabetes mellitus', 'Mean Corpuscular Hemoglobin Blood Test', 'Abnormal results of liver function studies', 'Vitamin D Blood Test','Hematocrit Blood Test','Mean corpuscular volume Blood Test','Other specified disorders involving the immune mechanism','Most recent BMI before MASLD diagnosis','Platelet Blood Test','Cardiac Risk Ratio','Red blood cell count Blood Test','Vitamin D deficiency, unspecified','Other nonspecific abnormal finding of lung field','Other specified counseling' ]
shap_values_top = shap_values_squeezed[:, :20]
x_test_top = x_test[:, :20]
# features_top = features[:20]
features_top = X.columns[:20]

plt.figure(figsize=(18, 7))
shap.summary_plot(shap_values_top, x_test_top, feature_names=features_top, show=False, max_display=10)
plt.xlabel('SHAP value (impact on model output)')
plt.title('Cox PH DeepSurv Influential Features')
# plt.savefig('results/coxph_shap.png', dpi=300)

In [ ]:
# Compute mean absolute SHAP values per feature
shap_mean_abs = np.abs(shap_values_squeezed).mean(axis=0)

# Create a Series for easy sorting
shap_series = pd.Series(shap_mean_abs, index=X.columns)

# Get top 20 feature names
top20_features = shap_series.sort_values(ascending=False).head(20).index.tolist()
top20_features

In [ ]:
# top features according to shap score
# rank features by mean absolute SHAP value
feature_names = X.columns

mean_abs_shap = np.abs(shap_values_squeezed).mean(axis=0)

top_indices = np.argsort(mean_abs_shap)[::-1]  # descending order

top_features = [(feature_names[i], mean_abs_shap[i]) for i in top_indices]

In [ ]:
# 1. Compute the mean SHAP value for each feature (not absolute)
mean_shap = shap_values_squeezed.mean(axis=0)

# 2. Get indices of top 10 positive and top 10 negative impact features
top_positive_indices = np.argsort(mean_shap)[-10:]  # most positive
top_negative_indices = np.argsort(mean_shap)[:10]   # most negative

# 3. Retrieve feature names and their SHAP values
top_positive_features = [(feature_names[i], mean_shap[i]) for i in reversed(top_positive_indices)]
top_negative_features = [(feature_names[i], mean_shap[i]) for i in top_negative_indices]

pos_names = [x[0] for x in top_positive_features]

neg_names = [x[0] for x in top_negative_features]

# Plot survival curves of populations stratified by certain features

In [ ]:
# i can only plot the survival estimates for the x_test
y_test_id = y_test.reset_index(inplace=False)
pos_outcome = y_test_studyid.index[X_test_scaled['Lab_2091-7'] > 0].tolist()
neg_outcome = y_test_studyid.index[X_test_scaled['Lab_2091-7'] <= 0].tolist()

pos_surv = surv[pos_outcome]
pos_surv_avg = pos_surv.mean(axis=1)

plt.plot(pos_surv_avg, label='Patients with Positive Lab_2091-7')
plt.plot(neg_surv_avg, label='Censored Patients')

plt.ylabel('Average S(t | x)')
_ = plt.xlabel('Time')
plt.title('Average Survival Prediction')
plt.legend()
# plt.savefig("average_survival.png", dpi=300)

In [ ]:
target_ids = X_test.index[X_test['Code_R94.5'] == True].tolist()

studyid_to_col = {study_id: i for i, study_id in enumerate(X_test.index)}
target_cols = [studyid_to_col[sid] for sid in target_ids if sid in studyid_to_col]
pos_surv = surv.iloc[:, target_cols]

pos_surv_avg = pos_surv.mean(axis=1)
neg_surv_avg = neg_surv.mean(axis=1)

plt.plot(pos_surv_avg, label='Patients Diagnosed with Abnormal Liver Function (n=609)')
plt.plot(neg_surv_avg, label='Patients Diagnosed without Abnormal Liver Function (n=2866)')

plt.ylabel('Average S(t | x)')
_ = plt.xlabel('Time t (days)')
plt.title('Predicted Time to Progression Stratified by Abnormal Liver Diagnosis')
plt.legend()
plt.savefig("results/coxph_abnormalliver.png", dpi=300)